In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math

In [2]:
train_df = pd.read_csv('data/processed/train_FD001_processed.csv')
test_df = pd.read_csv('data/processed/test_FD001_processed.csv')

In [3]:
def create_sliding_windows(df, lookback=30):
    # no comments, you can figure it out 👺
    X, y = [], []

    for engine_id in df["engine"].unique():
        engine_data = df[df["engine"] == engine_id].reset_index(drop=True)

        for i in range(len(engine_data) - lookback + 1):
            window = engine_data.iloc[i:i + lookback]
            X.append(window.drop(columns=["engine", "cycle", "RUL"]).values.flatten())
            y.append(window["RUL"].iloc[-1])
    return np.array(X), np.array(y)

In [4]:
LOOKBACK = 30
X_train, y_train = create_sliding_windows(train_df, lookback=LOOKBACK)

In [8]:
def get_last_window_per_engine(df, lookback=30):
    X_test, y_test = [], []

    for engine_id in df["engine"].unique():
        engine_data = df[df["engine"] == engine_id].reset_index(drop=True)

        if len(engine_data) >= lookback:
            window = engine_data.iloc[-lookback:]
            X_test.append(window.drop(columns=["engine", "cycle", "RUL"]).values.flatten())
            y_test.append(window["RUL"].iloc[-1])
        else:
            continue

    return np.array(X_test), np.array(y_test)

In [9]:
X_test, y_test = get_last_window_per_engine(test_df, lookback=LOOKBACK)

In [10]:
model = RandomForestRegressor(
    n_estimators=200,
    max_depth=20,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

,n_estimators,200
,criterion,'squared_error'
,max_depth,20
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [11]:
y_pred = model.predict(X_test)

In [12]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("Test MAE:", mae)
print("Test RMSE:", rmse)

for i, (true, pred) in enumerate(zip(y_test, y_pred)):
    print(f"Engine {i+1:03d} => True RUL = {true}, Predicted = {pred:.1f}")

Test MAE: 18.91832570940426
Test RMSE: 25.367978641597205
Engine 001 => True RUL = 112, Predicted = 156.9
Engine 002 => True RUL = 98, Predicted = 130.7
Engine 003 => True RUL = 69, Predicted = 57.0
Engine 004 => True RUL = 82, Predicted = 104.7
Engine 005 => True RUL = 91, Predicted = 114.5
Engine 006 => True RUL = 93, Predicted = 122.8
Engine 007 => True RUL = 91, Predicted = 123.7
Engine 008 => True RUL = 95, Predicted = 117.8
Engine 009 => True RUL = 111, Predicted = 132.6
Engine 010 => True RUL = 96, Predicted = 116.0
Engine 011 => True RUL = 97, Predicted = 81.1
Engine 012 => True RUL = 124, Predicted = 107.5
Engine 013 => True RUL = 95, Predicted = 95.2
Engine 014 => True RUL = 107, Predicted = 125.1
Engine 015 => True RUL = 83, Predicted = 151.9
Engine 016 => True RUL = 84, Predicted = 140.5
Engine 017 => True RUL = 50, Predicted = 57.5
Engine 018 => True RUL = 28, Predicted = 67.5
Engine 019 => True RUL = 87, Predicted = 126.2
Engine 020 => True RUL = 16, Predicted = 13.0
Engi